In [161]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [162]:
import pandas as pd
import numpy as np
import re
import nltk
import string
import random
import seaborn as sns
import pickle

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer, LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report, accuracy_score, hamming_loss, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [163]:
flightData = pd.read_csv('/content/drive/MyDrive/UCRIVERSIDE_20220617 (1).xlsx - EVENTS.csv')
flightData

,INDEX,REPORT DATE,NARRATIVE REDACTED,EVENT TYPE (ADAB CLASSIFICATION),CONFLICT (Y/N) (EX TERRAIN),PHASE 1,PHASE 2,PHASE 3,"1 AIRCRAFT DEPARTING, 1 AIRCRAFT CROSSING OR EXITING SAME RUNWAY","1 AIRCRAFT LANDING, 1 AIRCRAFT CROSSING OR EXITING SAME RUNWAY",...,TAXIED TO RUNWAY WITHOUT CLEARANCE,TCAS ALERT / RA,TCAS RA,TOUCH AND GO WRONG RUNWAY,TRAINING IN PROGRESS,TURBULENCE,UNMANNED AIRCRAFT SYSTEM,VEHICLE ON RUNWAY WITHOUT AUTHORIZATION,WAKE TURBULENCE,"WORKLOAD, DISTRACTION"
0,1,1/2/2014,"SIGNIFICANT OCCURRENCE REPORTED WHEN <N_1>, WM...",SIGNIFICANT / NOTEWORTHY OPERATIONAL OCCURRENCES,Y,LANDING,UNKNOWN,NaN,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,1/2/2014,"NMAC REPORTED WHEN <C_a>, B748, AND <C_b>, B74...",NEAR MID-AIR COLLISION (NMAC) REPORTS,Y,"UNKNOWN, AIRBORNE","UNKNOWN, AIRBORNE",NaN,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,1/2/2014,"<C_a>, PIPER PA31, DESCENDED 700 FEET BELOW AS...",PRELIMINARY REPORTS OF PILOT DEVIATIONS,N,"UNKNOWN, AIRBORNE",NaN,NaN,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,1/2/2014,"<N_1>, PIPER PA46, CLEARED TO LAND RUNWAY 13L...",PRELIMINARY REPORTS OF PILOT DEVIATIONS,N,LANDING,NaN,NaN,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1/2/2014,"<CS_fr>, GULFSTREAM GLF5, CLIMBED 500 FEET AB...",PRELIMINARY REPORTS OF PILOT DEVIATIONS,Y,"UNKNOWN, AIRBORNE","UNKNOWN, AIRBORNE",NaN,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21243,44822,12/28/2021,PEDESTRIAN ENTERED TWY Z AND Y WITHOUT AUTHORI...,AIRPORT ENVIRONMENT INCIDENTS,N,VEHICLE / PEDESTRIAN MOVEMENT,NaN,NaN,0,0,...,0,0,0,0,0,0,0,0,0,0
21244,44834,12/29/2021,"<N_1>, CESSNA C172, CROSSED RUNWAY 12L WITHOUT...",PRELIMINARY REPORTS OF PILOT DEVIATIONS,N,TAXI,NaN,NaN,0,0,...,0,0,0,0,0,0,0,0,0,0
21245,44837,12/29/2021,"<N_1>, CESSNA P210, CROSSED RUNWAY 22R HOLD LI...",PRELIMINARY REPORTS OF PILOT DEVIATIONS,N,TAXI,NaN,NaN,0,0,...,0,0,0,0,0,0,0,0,0,0
21246,44854,12/30/2021,"<CS_c>, A21N, ENTERED TWY B, VEHICLE ACCESS RO...",PRELIMINARY REPORTS OF PILOT DEVIATIONS,N,TAXI,NaN,NaN,0,0,...,0,0,0,0,0,0,0,0,0,0


In [164]:
flightData.shape

(21248, 132)

In [165]:
flightData['EVENT TYPE (ADAB CLASSIFICATION)'].unique()

array(['SIGNIFICANT / NOTEWORTHY OPERATIONAL OCCURRENCES',
       'NEAR MID-AIR COLLISION (NMAC) REPORTS',
       'PRELIMINARY REPORTS OF PILOT DEVIATIONS',
       'AIRPORT ENVIRONMENT INCIDENTS', 'MISCELLANEOUS',
       'NONCOMMERCIAL FATAL ACCIDENTS', 'COMMERCIAL FATAL ACCIDENTS',
       'ACCIDENT (FATAL UNK)', 'ILLNESS, INJURY', 'OPERATIONAL INCIDENT',
       'FOLLOW-UP INFO', 'NTSB ACTIVITY'], dtype=object)

In [166]:
labelBinarizer = LabelBinarizer()
labelBinarizer.fit(flightData['EVENT TYPE (ADAB CLASSIFICATION)'])
y = labelBinarizer.transform(flightData['EVENT TYPE (ADAB CLASSIFICATION)'])
labelBinarizer.classes_

array(['ACCIDENT (FATAL UNK)', 'AIRPORT ENVIRONMENT INCIDENTS',
       'COMMERCIAL FATAL ACCIDENTS', 'FOLLOW-UP INFO', 'ILLNESS, INJURY',
       'MISCELLANEOUS', 'NEAR MID-AIR COLLISION (NMAC) REPORTS',
       'NONCOMMERCIAL FATAL ACCIDENTS', 'NTSB ACTIVITY',
       'OPERATIONAL INCIDENT', 'PRELIMINARY REPORTS OF PILOT DEVIATIONS',
       'SIGNIFICANT / NOTEWORTHY OPERATIONAL OCCURRENCES'], dtype='<U48')

In [167]:
pd.DataFrame(y, columns = labelBinarizer.classes_)

,ACCIDENT (FATAL UNK),AIRPORT ENVIRONMENT INCIDENTS,COMMERCIAL FATAL ACCIDENTS,FOLLOW-UP INFO,"ILLNESS, INJURY",MISCELLANEOUS,NEAR MID-AIR COLLISION (NMAC) REPORTS,NONCOMMERCIAL FATAL ACCIDENTS,NTSB ACTIVITY,OPERATIONAL INCIDENT,PRELIMINARY REPORTS OF PILOT DEVIATIONS,SIGNIFICANT / NOTEWORTHY OPERATIONAL OCCURRENCES
0,0,0,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
21243,0,1,0,0,0,0,0,0,0,0,0,0
21244,0,0,0,0,0,0,0,0,0,0,1,0
21245,0,0,0,0,0,0,0,0,0,0,1,0
21246,0,0,0,0,0,0,0,0,0,0,1,0


In [168]:
'''
flightData['EVENT TYPE (ADAB CLASSIFICATION)'] = pd.factorize(flightData['EVENT TYPE (ADAB CLASSIFICATION)'])[0]

flightData['CONFLICT (Y/N) (EX TERRAIN)'] = pd.factorize(flightData['CONFLICT (Y/N) (EX TERRAIN)'])[0]

flightData['PHASE 1'] = pd.factorize(flightData['PHASE 1'])[0]

flightData['PHASE 2'] = pd.factorize(flightData['PHASE 2'])[0]

flightData['PHASE 3'] = pd.factorize(flightData['PHASE 3'])[0]
'''

"\nflightData['EVENT TYPE (ADAB CLASSIFICATION)'] = pd.factorize(flightData['EVENT TYPE (ADAB CLASSIFICATION)'])[0]\n\nflightData['CONFLICT (Y/N) (EX TERRAIN)'] = pd.factorize(flightData['CONFLICT (Y/N) (EX TERRAIN)'])[0]\n\nflightData['PHASE 1'] = pd.factorize(flightData['PHASE 1'])[0]\n\nflightData['PHASE 2'] = pd.factorize(flightData['PHASE 2'])[0]\n\nflightData['PHASE 3'] = pd.factorize(flightData['PHASE 3'])[0]\n"

In [169]:
flightData['EVENT TYPE (ADAB CLASSIFICATION)'].unique()

array(['SIGNIFICANT / NOTEWORTHY OPERATIONAL OCCURRENCES',
       'NEAR MID-AIR COLLISION (NMAC) REPORTS',
       'PRELIMINARY REPORTS OF PILOT DEVIATIONS',
       'AIRPORT ENVIRONMENT INCIDENTS', 'MISCELLANEOUS',
       'NONCOMMERCIAL FATAL ACCIDENTS', 'COMMERCIAL FATAL ACCIDENTS',
       'ACCIDENT (FATAL UNK)', 'ILLNESS, INJURY', 'OPERATIONAL INCIDENT',
       'FOLLOW-UP INFO', 'NTSB ACTIVITY'], dtype=object)

In [170]:
flightData

,INDEX,REPORT DATE,NARRATIVE REDACTED,EVENT TYPE (ADAB CLASSIFICATION),CONFLICT (Y/N) (EX TERRAIN),PHASE 1,PHASE 2,PHASE 3,"1 AIRCRAFT DEPARTING, 1 AIRCRAFT CROSSING OR EXITING SAME RUNWAY","1 AIRCRAFT LANDING, 1 AIRCRAFT CROSSING OR EXITING SAME RUNWAY",...,TAXIED TO RUNWAY WITHOUT CLEARANCE,TCAS ALERT / RA,TCAS RA,TOUCH AND GO WRONG RUNWAY,TRAINING IN PROGRESS,TURBULENCE,UNMANNED AIRCRAFT SYSTEM,VEHICLE ON RUNWAY WITHOUT AUTHORIZATION,WAKE TURBULENCE,"WORKLOAD, DISTRACTION"
0,1,1/2/2014,"SIGNIFICANT OCCURRENCE REPORTED WHEN <N_1>, WM...",SIGNIFICANT / NOTEWORTHY OPERATIONAL OCCURRENCES,Y,LANDING,UNKNOWN,NaN,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,1/2/2014,"NMAC REPORTED WHEN <C_a>, B748, AND <C_b>, B74...",NEAR MID-AIR COLLISION (NMAC) REPORTS,Y,"UNKNOWN, AIRBORNE","UNKNOWN, AIRBORNE",NaN,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,1/2/2014,"<C_a>, PIPER PA31, DESCENDED 700 FEET BELOW AS...",PRELIMINARY REPORTS OF PILOT DEVIATIONS,N,"UNKNOWN, AIRBORNE",NaN,NaN,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,1/2/2014,"<N_1>, PIPER PA46, CLEARED TO LAND RUNWAY 13L...",PRELIMINARY REPORTS OF PILOT DEVIATIONS,N,LANDING,NaN,NaN,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1/2/2014,"<CS_fr>, GULFSTREAM GLF5, CLIMBED 500 FEET AB...",PRELIMINARY REPORTS OF PILOT DEVIATIONS,Y,"UNKNOWN, AIRBORNE","UNKNOWN, AIRBORNE",NaN,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21243,44822,12/28/2021,PEDESTRIAN ENTERED TWY Z AND Y WITHOUT AUTHORI...,AIRPORT ENVIRONMENT INCIDENTS,N,VEHICLE / PEDESTRIAN MOVEMENT,NaN,NaN,0,0,...,0,0,0,0,0,0,0,0,0,0
21244,44834,12/29/2021,"<N_1>, CESSNA C172, CROSSED RUNWAY 12L WITHOUT...",PRELIMINARY REPORTS OF PILOT DEVIATIONS,N,TAXI,NaN,NaN,0,0,...,0,0,0,0,0,0,0,0,0,0
21245,44837,12/29/2021,"<N_1>, CESSNA P210, CROSSED RUNWAY 22R HOLD LI...",PRELIMINARY REPORTS OF PILOT DEVIATIONS,N,TAXI,NaN,NaN,0,0,...,0,0,0,0,0,0,0,0,0,0
21246,44854,12/30/2021,"<CS_c>, A21N, ENTERED TWY B, VEHICLE ACCESS RO...",PRELIMINARY REPORTS OF PILOT DEVIATIONS,N,TAXI,NaN,NaN,0,0,...,0,0,0,0,0,0,0,0,0,0


In [171]:
stop_words = stopwords.words('english')

def remove_stop_words(narrative):
  return ' '.join([word for word in narrative.split() if word not in stop_words and len(word) > 1])

In [172]:
narratives = flightData['NARRATIVE REDACTED']

# Remove all tags from narratives
narratives = [re.sub(r'<(.+?)>','',narrative) for narrative in narratives]

# Stripping all punctuation from narratives
narratives = [narrative.translate(str.maketrans('','',string.punctuation)) for narrative in narratives]

# Remove all stopwords from narratives
narratives = list(map(remove_stop_words,narratives))
narratives = np.array(narratives)

In [173]:
tfidf = TfidfVectorizer(min_df=0.05,max_df=0.95,stop_words=stop_words)
y = flightData['EVENT TYPE (ADAB CLASSIFICATION)']
x_train, x_test, y_train, y_test = train_test_split(narratives, y, test_size=0.4)
X_train = tfidf.fit_transform(x_train)
X_test = tfidf.transform(x_test)

In [174]:
# clf = LinearSVC()
clf = LogisticRegression(max_iter=10000).fit(X_train, y_train)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.9352941176470588

In [175]:
y_pred = clf.predict(X_test)
# print(classification_report(y_test, y_pred))
print(f"Test Set Accuracy : {accuracy_score(y_test, y_pred) * 100} %\n\n")
print(f"Test Set Loss : {hamming_loss(y_test, y_pred) * 100} %\n\n")
print(f"Classification Report : \n\n{classification_report(y_test, y_pred)}")

Test Set Accuracy : 93.52941176470588 %


Test Set Loss : 6.470588235294119 %


Classification Report : 

                                                  precision    recall  f1-score   support

                   AIRPORT ENVIRONMENT INCIDENTS       0.95      0.83      0.89      1632
                      COMMERCIAL FATAL ACCIDENTS       0.00      0.00      0.00         6
                                 ILLNESS, INJURY       0.00      0.00      0.00         3
                                   MISCELLANEOUS       0.66      0.86      0.74       249
           NEAR MID-AIR COLLISION (NMAC) REPORTS       0.98      0.92      0.95       301
                   NONCOMMERCIAL FATAL ACCIDENTS       0.87      0.65      0.74       276
                                   NTSB ACTIVITY       0.00      0.00      0.00         1
                            OPERATIONAL INCIDENT       0.56      0.33      0.42        42
         PRELIMINARY REPORTS OF PILOT DEVIATIONS       0.94      0.99      0.97    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [176]:
#pickle.dump(clf, open('EventType_model.pkl', 'wb'))
#pickle.dump(tfidf, open('EventType_vectorizer.pkl', 'wb'))

In [177]:
def infer_event(x):
    x = remove_stop_words(x)
    x = re.sub(r'<(.+?)>','',x)
    x_vec = tfidf.transform([x])
    x_pred = clf.predict(x_vec)
    return x_pred
    #return labelBinarizer.inverse_transform(x_pred)

In [178]:
for i in range(5):
  k = random.sample(list(enumerate(x_test)), 1)
  print("Narrative: ", flightData['NARRATIVE REDACTED'][k[0][0]])
  print("\nPredicted categories: ", infer_event(k[0][1]))
  print("Actual categories: ", flightData['EVENT TYPE (ADAB CLASSIFICATION)'][k[0][0]])
  print()


Narrative:  <N_1>, CIRRUS SR22, DEPARTED RUNWAY 12R WITHOUT CLEARANCE CONFLICTING WITH <CS_i>, A320, ON LANDING ROLL SAME RUNWAY
SAN ANTONIO, TX 8/27 1034C.  CLOSEST PROXIMITY 500-1,000 FEET LATERAL.  

Predicted categories:  ['PRELIMINARY REPORTS OF PILOT DEVIATIONS']
Actual categories:  AIRPORT ENVIRONMENT INCIDENTS

Narrative:  <N_1>, AEROSPATIALE AS35, REPORTED NMAC WITH 2 UNIDENTIFIED ACFT AT 2,000 FEET 20 S NORMAN, OK 211615C. <N_1> DESCENDED TO AVOID UNIDENTIFIED ACFTS. CLOSEST PROXIMITY 100 FEET VERTICAL, 100 FEET LATERAL.  

Predicted categories:  ['PRELIMINARY REPORTS OF PILOT DEVIATIONS']
Actual categories:  NEAR MID-AIR COLLISION (NMAC) REPORTS

Narrative:  <MISC_1>, HAWKER BEECHCRAFT H25B, CROSSED RUNWAY 27L HOLD LINE WITHOUT CLEARANCE CONFLICTING WITH <CS_c>, A321,
CHICAGO, IL 12/30 0940C.  <CS_c> went around to avoid loss of sepn.  

Predicted categories:  ['PRELIMINARY REPORTS OF PILOT DEVIATIONS']
Actual categories:  PRELIMINARY REPORTS OF PILOT DEVIATIONS

Narrative: 

In [181]:
narr = pd.Series(x_test)
classified = pd.Series(y_test.values)

narr.name = 'NARRATIVE REDACTED'
classified.name = 'EVENT TYPE (ADAB CLASSIFICATION)'

act_vs_pred_events = pd.concat([narr, classified], axis = 1)
act_vs_pred_events["Predicted"] = act_vs_pred_events.apply(lambda x: infer_event(x['NARRATIVE REDACTED']), axis=1)

In [182]:
act_vs_pred_events.to_excel('Actual_vs_Predicted_Events.xlsx')